In [ ]:
# Load the dataset
df = pd.read_csv('macroeconomic_data.csv', parse_dates=['Date'])

# Display the first few rows
print("First few rows of the dataset:")
print(df.head())

In [ ]:

# Check for missing values
print("\nMissing values in the dataset:")
print(df.isnull().sum())

# Describe the dataset
print("\nStatistical description of the dataset:")
print(df.describe())

In [ ]:
# Plotting the time series of each feature
plt.figure(figsize=(14, 8))
for column in df.columns[1:]:
    plt.plot(df['Date'], df[column], label=column)
plt.title('Macroeconomic Indicators Over Time')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
# Plot correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# Plot histograms of each feature
df.hist(bins=30, figsize=(14, 10))
plt.suptitle('Histogram of Features')
plt.show()


In [ ]:
# Plot pairplot to see the relationships between variables
sns.pairplot(df.drop('Date', axis=1))
plt.suptitle('Pairplot of Features', y=1.02)
plt.show()

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Normalize the features
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df.drop('Date', axis=1)), columns=df.columns[1:])

# Prepare the data for time series forecasting
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        label = data[i + seq_length]
        sequences.append((seq, label))
    return sequences

seq_length = 12  # Sequence length of 12 months
sequences = create_sequences(df_scaled.values, seq_length)

# Split the data into training and testing sets
train_size = int(len(sequences) * 0.8)
train_sequences = sequences[:train_size]
test_sequences = sequences[train_size:]

# Separate features and labels
X_train = np.array([seq[0] for seq in train_sequences])
y_train = np.array([seq[1] for seq in train_sequences])
X_test = np.array([seq[0] for seq in test_sequences])
y_test = np.array([seq[1] for seq in test_sequences])


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define the LSTM model
lstm_model = Sequential([
    LSTM(50, activation='relu', input_shape=(seq_length, X_train.shape[2])),
    Dense(y_train.shape[1])
])

lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.summary()

# Train the LSTM model
history_lstm = lstm_model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                11400     
                                                                 
 dense (Dense)               (None, 6)                 306       
                                                                 
Total params: 11706 (45.73 KB)
Trainable params: 11706 (45.73 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
25/25 [==============================] - 2s 19ms/step - loss: 0.2597 - val_loss: 0.1286
Epoch 2/50
25/25 [==============================] - 0s 8ms/step - loss: 0.0967 - val_loss: 0.0884
Epoch 3/50
25/25 [==============================] - 0s 10ms/step - loss: 0.0881 - val_loss: 0.0877
Epoch 4/50
25/25 [==============================] - 0s 9ms/step - loss: 0.0873 - val_loss: 0.0875
Epoch 5/50
25/25 [====

In [4]:
from tensorflow.keras.layers import GRU

# Define the GRU model
gru_model = Sequential([
    GRU(50, activation='relu', input_shape=(seq_length, X_train.shape[2])),
    Dense(y_train.shape[1])
])

gru_model.compile(optimizer='adam', loss='mse')
gru_model.summary()

# Train the GRU model
history_gru = gru_model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 50)                8700      
                                                                 
 dense_1 (Dense)             (None, 6)                 306       
                                                                 
Total params: 9006 (35.18 KB)
Trainable params: 9006 (35.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
25/25 [==============================] - 2s 19ms/step - loss: 0.1732 - val_loss: 0.0876
Epoch 2/50
25/25 [==============================] - 0s 9ms/step - loss: 0.0906 - val_loss: 0.0887
Epoch 3/50
25/25 [==============================] - 0s 10ms/step - loss: 0.0880 - val_loss: 0.0867
Epoch 4/50
25/25 [==============================] - 0s 9ms/step - loss: 0.0872 - val_loss: 0.0861
Epoch 5/50
25/25 [====

In [5]:
from tensorflow.keras.layers import Flatten

# Define the DNN model
dnn_model = Sequential([
    Flatten(input_shape=(seq_length, X_train.shape[2])),
    Dense(100, activation='relu'),
    Dense(y_train.shape[1])
])

dnn_model.compile(optimizer='adam', loss='mse')
dnn_model.summary()

# Train the DNN model
history_dnn = dnn_model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 72)                0         
                                                                 
 dense_2 (Dense)             (None, 100)               7300      
                                                                 
 dense_3 (Dense)             (None, 6)                 606       
                                                                 
Total params: 7906 (30.88 KB)
Trainable params: 7906 (30.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
25/25 [==============================] - 1s 8ms/step - loss: 0.1807 - val_loss: 0.1320
Epoch 2/50
25/25 [==============================] - 0s 4ms/step - loss: 0.1154 - val_loss: 0.1195
Epoch 3/50
25/25 [==============================] - 0s 6ms/step - loss: 0.1047 - val_los